# Idea
Do some training

In [1]:
SEED = 133
TIME_WINDOW = "30s"
DATA_FILE = '../../data/all_df_30s_Xi_sensor_1.csv'
DATA_FILE_2 = '../../data/all_df_30s_Heqing_device2.csv'

COLUMNS_SELECTION_FILE_FROM_RP4 = "../other_time_windows/weight_selected_30s.csv"

In [2]:
from pathlib import Path

import numpy as np
import pandas as pd
import pickle

assert Path(DATA_FILE).exists()

In [3]:
import sys
repo_base_path = Path("./../../../").resolve()
assert str(repo_base_path).endswith("csg_is"), f"{repo_base_path} is not a valid path to the CSG_IS repository" 

sys.path.append(str(repo_base_path))
sys.path.append("..")

# from py_dataset import get_all_files_df
# from py_dataset import read_in_files
from py_dataset import feature_plotting
# from py_dataset import feature_selection
# from py_dataset import net_feature_engineering
# from py_dataset import entropy_feature_engineering
# from py_dataset.classes import DataSource
from py_dataset import sys_func

from py_dataset.classes import DataSource



In [4]:
LOG_PATH = (repo_base_path / "training" / "official_logs")
LOG_NAME = f"single-layer_300_{TIME_WINDOW}_Xi_sensor_1_and_Heqing_device2"
LOG_NAME_EXPERIMENT_2 = f"single-layer_30_from_rp4_weights_{TIME_WINDOW}_Xi_sensor_1_and_Heqing_device2"
LOG_NAME_EXPERIMENT_3 = f"multi-layer_30_from_rp4_weights_{TIME_WINDOW}_Xi_sensor_1_and_Heqing_device2"

assert LOG_PATH.exists()
assert not (LOG_PATH / LOG_NAME).exists(), "are you sure to run this again?"
assert not (LOG_PATH / LOG_NAME_EXPERIMENT_2).exists(), "are you sure to run this again?"
assert not (LOG_PATH / LOG_NAME_EXPERIMENT_3).exists(), "are you sure to run this again?"

In [5]:
df1 = pd.read_csv(DATA_FILE)
df1.set_index("timestamp", inplace=True)

/tmp/ipykernel_15790/3829008155.py:1: DtypeWarning: Columns (189) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv(DATA_FILE)


In [6]:
df2 = pd.read_csv(DATA_FILE_2)
df2.set_index("timestamp", inplace=True)

In [7]:
df = pd.concat([df1,df2])
df.shape

(67555, 306)

In [8]:
assert df["label"].unique().shape == (10,)

In [13]:
selected_columns = pd.read_csv(COLUMNS_SELECTION_FILE_FROM_RP4)["0"].values
set(df2.columns) - set(df1.columns)

{'armv7_cortex_a15/br_mis_pred/',
 'armv7_cortex_a15/br_pred/',
 'armv7_cortex_a15/bus_cycles/',
 'armv7_cortex_a15/bus_cycles/.1',
 'armv7_cortex_a15/cpu_cycles/',
 'armv7_cortex_a15/exc_return/',
 'armv7_cortex_a15/exc_taken/',
 'armv7_cortex_a15/l1d_cache/',
 'armv7_cortex_a15/l1d_cache_refill/',
 'armv7_cortex_a15/l1d_tlb_refill/',
 'armv7_cortex_a15/l1i_cache/',
 'armv7_cortex_a15/l1i_cache_refill/',
 'armv7_cortex_a15/l1i_tlb_refill/',
 'armv7_cortex_a15/l2d_cache/',
 'armv7_cortex_a15/l2d_cache_wb/',
 'armv7_cortex_a15/mem_access/',
 'gettimeofday',
 'open'}

### Setup

In [11]:
from torch.utils.data import Dataset, DataLoader
import torch

class CustomeDataset(Dataset):
    def __init__(self, some_df:pd.DataFrame):
        
        self.x = torch.tensor(some_df.drop(columns=["label"], inplace=False).values, dtype=torch.float32)
        self.labels = torch.tensor(some_df["label"].values, dtype=torch.uint8)

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        # if torch.is_tensor(idx):
        #     idx = idx.tolist()
        # print(self.x[idx], self.labels[idx])
        return self.x[idx], self.labels[idx]

In [12]:
from mlp_single_layer import MalwaresModelLinearLayer
from torch.utils.data import DataLoader
import lightning
from sklearn.model_selection import train_test_split
from lightning import loggers as pl_loggers
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from lightning.pytorch.callbacks.model_checkpoint import ModelCheckpoint

# tensorboard = pl_loggers.TensorBoardLogger(save_dir=LOG_PATH, name=LOG_NAME)

# # split into train and val
# train, val = train_test_split(vectors, test_size=0.2, random_state=SEED)

# train_dataset = CustomeDataset(train)
# train_dataloader = DataLoader(train_dataset, batch_size=500, shuffle=True, num_workers=7)

# val_dataset = CustomeDataset(val)
# val_dataloader = DataLoader(val_dataset, batch_size=500, shuffle=False, num_workers=7)

# # trainer = lightning.Trainer(fast_dev_run=100)
# monitor = "Validation/Accuracy"
# early_stop_callback = EarlyStopping(monitor=monitor, patience=5, verbose=True, mode="max")
# checkpoint_callback = ModelCheckpoint(save_top_k=1, monitor=monitor, mode="max")

# trainer = lightning.Trainer(max_epochs=150, callbacks=[early_stop_callback, checkpoint_callback], enable_checkpointing=True, logger=tensorboard) #logger=None, 


# model = MalwaresModelLinearLayer(input_size = vectors.shape[1]-1, output_size = len(label_encoder.classes_), seed=SEED)

# torch.manual_seed(SEED)
# torch.cuda.manual_seed_all(SEED)


In [ ]:
# trainer.fit(model=model, train_dataloaders=train_dataloader, val_dataloaders=val_dataloader)

In [ ]:
# model.eval()

In [ ]:
# print(checkpoint_callback.best_model_path)   # prints path to the best model's checkpoint
# print(checkpoint_callback.best_model_score) # and prints it score
# best_model = MalwaresModelLinearLayer.load_from_checkpoint(checkpoint_callback.best_model_path, input_size = vectors.shape[1]-1, output_size = len(label_encoder.classes_))

# trainer.test(best_model, dataloaders=val_dataloader)

### Preprocessings
- remove Null values
- drop timestamp
- labelEncode
- normalize

In [29]:
# select here df1 => remove cols from rp4 that are not in rp3
print ("going to remove columns: ", set(selected_columns) - set(df1.columns))
selected_columns_that_are_in_rp3_dataset = list(set(selected_columns) & set(df1.columns))

going to remove columns:  {'armv7_cortex_a15/br_mis_pred/'}


In [30]:
df[selected_columns_that_are_in_rp3_dataset].shape

(67555, 32)

In [31]:
subset = df[selected_columns_that_are_in_rp3_dataset].copy()
subset["label"] = df["label"]
subset.shape

(67555, 32)

In [33]:
print(subset.shape)
old_df = subset.copy()
subset.dropna(axis=0, how='any', inplace=True)

print(subset.shape)

(67555, 32)
(44286, 32)


In [34]:
subset.reset_index(inplace=True, drop=True)

In [35]:
label_encoder = sys_func.create_label_encoder_for_labels()
subset['label'] = label_encoder.transform(subset['label'])

In [36]:
subset = feature_plotting.test_transform_all_to_numeric_columns(subset)

In [37]:
cols_to_exclude_from_scaling = ["label"]
vectors = feature_plotting.minmax_scale_features(subset,remove_outliers_iqr_all_columns=False,
                                                        cols_to_exclude_from_scaling=cols_to_exclude_from_scaling)
# print(vectors.shape)
# print(" with dropped null: ", vectors.dropna(axis=0, how='any', inplace=False).shape)

In [38]:
subset["label"].unique(), vectors["label"].unique()

(array([0, 8, 4, 1, 5, 2, 7, 6, 3]), array([0, 8, 4, 1, 5, 2, 7, 6, 3]))

In [40]:
# split into train and val
train, val = train_test_split(vectors, test_size=0.2, random_state=SEED)

train_dataset = CustomeDataset(train)
train_dataloader = DataLoader(train_dataset, batch_size=500, shuffle=True, num_workers=7)

val_dataset = CustomeDataset(val)
val_dataloader = DataLoader(val_dataset, batch_size=500, shuffle=False, num_workers=7)

# trainer = lightning.Trainer(fast_dev_run=100)
monitor = "Validation/Accuracy"
early_stop_callback = EarlyStopping(monitor=monitor, patience=5, verbose=True, mode="max")
checkpoint_callback = ModelCheckpoint(save_top_k=1, monitor=monitor, mode="max")

tensorboard = pl_loggers.TensorBoardLogger(save_dir=LOG_PATH, name=LOG_NAME_EXPERIMENT_2)
trainer = lightning.Trainer(max_epochs=150, callbacks=[early_stop_callback, checkpoint_callback], enable_checkpointing=True, logger=tensorboard) #logger=None, 


model = MalwaresModelLinearLayer(input_size = vectors.shape[1]-1, output_size = len(label_encoder.classes_), seed=SEED)

torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [42]:
trainer.fit(model=model, train_dataloaders=train_dataloader, val_dataloaders=val_dataloader)

/home/<User>/repos/csg_is/.venv/lib/python3.10/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:653: Checkpoint directory /home/<User>/repos/csg_is/training/official_logs/single-layer_30_from_rp4_weights_30s_Xi_sensor_1_and_Heqing_device2/version_0/checkpoints exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type                      | Params
------------------------------------------------------------
0 | train_metrics | MetricCollection          | 0     
1 | val_metrics   | MetricCollection          | 0     
2 | test_metrics  | MetricCollection          | 0     
3 | cm            | MulticlassConfusionMatrix | 0     
4 | criterion     | CrossEntropyLoss          | 0     
5 | l1            | Linear                    | 288   
------------------------------------------------------------
288       Trainable params
0         Non-trainable params
288       Total params
0.001     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Epoch end: Validation, epoch number: 25
ValidationEpoch/CM
 tensor([[ 53668,     82,   3419,  52835,     93,     87,     24,      0,   2446],
        [     0, 114681,     90,      0,      1,    191,     50,      0,   1093],
        [  3774,    372, 137051,   4245,    170,   1064,   1413,     75,    352],
        [     0,      0,   5029, 108629,      0,    389,      8,     10,    239],
        [    92,    221,   2632,  38652,  72368,    239,    159,      2,    461],
        [     0,   1140,  14018,    100,      0,  96064,   2472,   1193,    185],
        [     2,      0,  31436,  23496,     19,   1322,  31054,  26073,    206],
        [     0,      2,    226,     99,      9,    121,  11287, 102762,      0],
        [  1812,    543,   2102,  39969,      4,      4,     54,      0,  70684]])


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Epoch end: Validation, epoch number: 84
ValidationEpoch/CM
 tensor([[256583,     82,   4368, 117858,    334,     87,   1161,      0,   8596],
        [     0, 399628,     91,      0,      1,    191,     50,      0,   1115],
        [  7720,    518, 482512,   4636,    170,   1681,  15295,     75,    352],
        [  5204,      0,   5922, 379455,    206,    389,   2098,     10,   1565],
        [   387,    221,   3339,  46202, 342852,    239,   1956,      2,   1294],
        [     0,   1239,  15774,    100,      0, 376751,   2540,   1193,    185],
        [  4353,      0, 101257,  53318,   1966,   2046, 166044,  61723,   1676],
        [     0,      2,    226,     99,      9,   1263,  35975, 357772,      0],
        [  5773,    543,   2630,  48958,    295,      4,    243,      0, 339336]])
Epoch end: Train, epoch number: 81
TrainEpoch/CM
 tensor([[256583,     82,   4368, 117858,    334,     87,   1161,      0,   8596],
        [     0, 399628,     91,      0,      1,    191,     50,     

Validation: |          | 0/? [00:00<?, ?it/s]

Epoch end: Validation, epoch number: 85
ValidationEpoch/CM
 tensor([[260698,     82,   4385, 118269,    334,     87,   1196,      0,   8703],
        [     0, 404458,     91,      0,      1,    191,     50,      0,   1115],
        [  7779,    518, 488400,   4636,    170,   1685,  15521,     75,    352],
        [  5332,      0,   5927, 384002,    206,    389,   2153,     10,   1585],
        [   392,    221,   3345,  46229, 347563,    239,   1976,      2,   1299],
        [     0,   1239,  15779,    100,      0, 381536,   2540,   1193,    185],
        [  4472,      0, 102220,  53694,   1988,   2051, 168750,  62228,   1705],
        [     0,      2,    226,     99,      9,   1276,  36289, 362205,      0],
        [  5831,    543,   2636,  49061,    300,      4,    248,      0, 343949]])
Epoch end: Train, epoch number: 82


Monitored metric Validation/Accuracy did not improve in the last 5 records. Best score: 0.916. Signaling Trainer to stop.


TrainEpoch/CM
 tensor([[260698,     82,   4385, 118269,    334,     87,   1196,      0,   8703],
        [     0, 404458,     91,      0,      1,    191,     50,      0,   1115],
        [  7779,    518, 488400,   4636,    170,   1685,  15521,     75,    352],
        [  5332,      0,   5927, 384002,    206,    389,   2153,     10,   1585],
        [   392,    221,   3345,  46229, 347563,    239,   1976,      2,   1299],
        [     0,   1239,  15779,    100,      0, 381536,   2540,   1193,    185],
        [  4472,      0, 102220,  53694,   1988,   2051, 168750,  62228,   1705],
        [     0,      2,    226,     99,      9,   1276,  36289, 362205,      0],
        [  5831,    543,   2636,  49061,    300,      4,    248,      0, 343949]])


In [43]:
print(checkpoint_callback.best_model_path)   # prints path to the best model's checkpoint
print(checkpoint_callback.best_model_score) # and prints it score
best_model = MalwaresModelLinearLayer.load_from_checkpoint(checkpoint_callback.best_model_path, input_size = vectors.shape[1]-1, output_size = len(label_encoder.classes_))

trainer.test(best_model, dataloaders=val_dataloader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


/home/<User>/repos/csg_is/training/official_logs/single-layer_30_from_rp4_weights_30s_Xi_sensor_1_and_Heqing_device2/version_0/checkpoints/epoch=78-step=5609.ckpt
tensor(0.9156, device='cuda:0')


Testing: |          | 0/? [00:00<?, ?it/s]

Epoch end: Test, epoch number: 0
TestEpoch/CM
 tensor([[ 838,    0,    5,   84,    0,    0,    7,    0,   23],
        [   0,  904,    0,    0,    0,    0,    0,    0,    0],
        [   9,    0, 1148,    0,    0,    1,   57,    0,    0],
        [  19,    0,    0,  877,    0,    0,    9,    0,    3],
        [   1,    0,    1,    3,  945,    0,    6,    0,    0],
        [   0,    0,    0,    0,    0,  997,    0,    0,    0],
        [  27,    0,  191,   76,    2,    2,  558,   99,    6],
        [   0,    0,    0,    0,    0,    1,   74,  930,    0],
        [  11,    0,    2,   25,    0,    0,    1,    0,  916]])
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      Test/Accuracy         0.9156419038772583
      Test/F1Score          0.9124975800514

[{'Test/Loss': 0.31683042645454407,
  'Test/Accuracy': 0.9156419038772583,
  'Test/Precision': 0.916144847869873,
  'Test/Recall': 0.9156419038772583,
  'Test/F1Score': 0.9124975800514221,
  'TestEpoch/Accuracy': 0.915560245513916,
  'TestEpoch/Precision': 0.9165933132171631,
  'TestEpoch/Recall': 0.915560245513916,
  'TestEpoch/F1Score': 0.9134067296981812}]

In [44]:
sys.path.append(str(repo_base_path  / "training"))

In [47]:
# tensorboard = pl_loggers.TensorBoardLogger('./')

from mlp import MalwaresModelMLP

# split into train and val
train, val = train_test_split(vectors, test_size=0.2, random_state=SEED)

train_dataset = CustomeDataset(train)
train_dataloader = DataLoader(train_dataset, batch_size=500, shuffle=True, num_workers=7)

val_dataset = CustomeDataset(val)
val_dataloader = DataLoader(val_dataset, batch_size=500, shuffle=False, num_workers=7)

# trainer = lightning.Trainer(fast_dev_run=100)
monitor = "Validation/Accuracy"
early_stop_callback = EarlyStopping(monitor=monitor, patience=5, verbose=True, mode="max")
checkpoint_callback = ModelCheckpoint(save_top_k=1, monitor=monitor, mode="max")

tensorboard = pl_loggers.TensorBoardLogger(save_dir=LOG_PATH, name=LOG_NAME_EXPERIMENT_3)
trainer = lightning.Trainer(max_epochs=150, callbacks=[early_stop_callback, checkpoint_callback], enable_checkpointing=True, logger=tensorboard) #logger=None, 


model = MalwaresModelMLP(input_size = vectors.shape[1]-1, output_size = len(label_encoder.classes_), seed=SEED)

torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [48]:

trainer.fit(model=model, train_dataloaders=train_dataloader, val_dataloaders=val_dataloader)


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type                      | Params
------------------------------------------------------------
0 | train_metrics | MetricCollection          | 0     
1 | val_metrics   | MetricCollection          | 0     
2 | test_metrics  | MetricCollection          | 0     
3 | cm            | MulticlassConfusionMatrix | 0     
4 | criterion     | CrossEntropyLoss          | 0     
5 | l1            | Linear                    | 960   
6 | l2            | Linear                    | 930   
7 | l3            | Linear                    | 279   
------------------------------------------------------------
2.2 K     Trainable params
0         Non-trainable params
2.2 K     Total params
0.009     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Epoch end: Validation, epoch number: 0
ValidationEpoch/CM
 tensor([[  0,   0, 107,   0,   0,   0,   0,   0,   0],
        [  0,   0,  83,   0,   0,   0,  10,   0,   0],
        [  0,   0, 134,   0,   0,   0,   0,   0,   0],
        [  0,   0,  92,   0,   0,   0,   0,   0,   0],
        [  0,   0, 125,   0,   0,   0,   0,   0,   0],
        [  0,   0, 106,   0,   0,   0,   0,   0,   0],
        [  0,   0,  99,   0,   0,   0,   5,   0,   0],
        [  0,   0,  75,   0,   0,   0,  58,   0,   0],
        [  0,   0, 106,   0,   0,   0,   0,   0,   0]])


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Epoch end: Validation, epoch number: 49
ValidationEpoch/CM
 tensor([[216356,      0,   1780,   7830,   1378,      1,    743,      2,   1582],
        [     0, 236304,    277,      0,     16,    148,     17,      0,      1],
        [  1026,     13, 289495,    728,    742,     95,  10532,     13,    163],
        [   994,      0,   2039, 224290,    128,      4,   5105,      1,    526],
        [   784,      0,   1315,   3264, 228395,      0,    265,      0,     28],
        [     0,    133,    489,      1,    219, 233130,    536,    304,      4],
        [   868,     68,  19535,  14640,     31,     22, 185509,  10885,     71],
        [     0,    182,    194,      0,      0,      0,   7558, 225439,      0],
        [  1617,      0,   1313,   5038,    349,     88,    115,      0, 226296]])
Epoch end: Train, epoch number: 48
TrainEpoch/CM
 tensor([[216356,      0,   1780,   7830,   1378,      1,    743,      2,   1582],
        [     0, 236304,    277,      0,     16,    148,     17,     

Validation: |          | 0/? [00:00<?, ?it/s]

Epoch end: Validation, epoch number: 50
ValidationEpoch/CM
 tensor([[221033,      0,   1781,   7830,   1378,      1,    743,      2,   1589],
        [     0, 241134,    277,      0,     16,    148,     17,      0,      1],
        [  1035,     13, 295534,    728,    743,     95,  10660,     13,    163],
        [   994,      0,   2039, 228955,    128,      4,   5195,      1,    526],
        [   784,      0,   1315,   3264, 233165,      0,    269,      0,     28],
        [     0,    133,    489,      1,    219, 237920,    536,    304,      4],
        [   868,     68,  19618,  14847,     32,     22, 189900,  10928,     71],
        [     0,    182,    194,      0,      0,      0,   7592, 230165,      0],
        [  1628,      0,   1313,   5038,    349,     88,    115,      0, 231075]])
Epoch end: Train, epoch number: 49


Monitored metric Validation/Accuracy did not improve in the last 5 records. Best score: 0.987. Signaling Trainer to stop.


TrainEpoch/CM
 tensor([[221033,      0,   1781,   7830,   1378,      1,    743,      2,   1589],
        [     0, 241134,    277,      0,     16,    148,     17,      0,      1],
        [  1035,     13, 295534,    728,    743,     95,  10660,     13,    163],
        [   994,      0,   2039, 228955,    128,      4,   5195,      1,    526],
        [   784,      0,   1315,   3264, 233165,      0,    269,      0,     28],
        [     0,    133,    489,      1,    219, 237920,    536,    304,      4],
        [   868,     68,  19618,  14847,     32,     22, 189900,  10928,     71],
        [     0,    182,    194,      0,      0,      0,   7592, 230165,      0],
        [  1628,      0,   1313,   5038,    349,     88,    115,      0, 231075]])


In [49]:

print(checkpoint_callback.best_model_path)   # prints path to the best model's checkpoint
print(checkpoint_callback.best_model_score) # and prints it score
best_model = MalwaresModelMLP.load_from_checkpoint(checkpoint_callback.best_model_path, input_size = vectors.shape[1]-1, output_size = len(label_encoder.classes_))

trainer.test(best_model, dataloaders=val_dataloader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


/home/<User>/repos/csg_is/training/official_logs/multi-layer_30_from_rp4_weights_30s_Xi_sensor_1_and_Heqing_device2/version_0/checkpoints/epoch=44-step=3195.ckpt
tensor(0.9868, device='cuda:0')


Testing: |          | 0/? [00:00<?, ?it/s]

Epoch end: Test, epoch number: 0
TestEpoch/CM
 tensor([[ 957,    0,    0,    0,    0,    0,    0,    0,    0],
        [   0,  904,    0,    0,    0,    0,    0,    0,    0],
        [   2,    0, 1199,    0,    0,    0,   14,    0,    0],
        [   0,    0,    0,  899,    0,    0,    9,    0,    0],
        [   0,    0,    0,    0,  956,    0,    0,    0,    0],
        [   0,    0,    0,    0,    0,  997,    0,    0,    0],
        [   0,    0,   24,   40,    0,    0,  886,   11,    0],
        [   0,    0,    0,    0,    0,    0,    5, 1000,    0],
        [   3,    0,    0,    5,    0,    2,    0,    0,  945]])
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      Test/Accuracy         0.9868288636207581
      Test/F1Score           0.986780047416

[{'Test/Loss': 0.04320938140153885,
  'Test/Accuracy': 0.9868288636207581,
  'Test/Precision': 0.9871439337730408,
  'Test/Recall': 0.9868288636207581,
  'Test/F1Score': 0.986780047416687,
  'TestEpoch/Accuracy': 0.9870476722717285,
  'TestEpoch/Precision': 0.9871102571487427,
  'TestEpoch/Recall': 0.9870476722717285,
  'TestEpoch/F1Score': 0.9869661331176758}]

In [ ]:
vectors.columns.to_frame().to_csv(f"weight_selected_{TIME_WINDOW}.csv", index=False)